# Using Fast and Powerful Full-text search Engine - Elasticsearch 

Setup Elasticsearch Cloud instance by going to -> [Elastic cloud](https://cloud.elastic.co)

## Load data set

In [1]:
import pandas as pd
from pathlib import Path

CORD19_PATH = Path('../data/input/trec_cord19_v0.csv')

def load_cord19(input_fpath: Path, dtype: str = 'csv', cols_to_keep: list = ['cord_uid', 'abstract'], index_col = 'cord_uid') -> pd.DataFrame:
    """Loads CORD19 data and returns it as pandas data frame
    """
    if dtype == 'csv':
        df = pd.read_csv(input_fpath, quotechar='"', index_col=index_col, usecols=cols_to_keep)
        # for each column
        for col in df.columns:
            # check if the columns contains string data
            if pd.api.types.is_string_dtype(df[col]):
                df[col] = df[col].str.strip() # removes front and end white spaces
                df[col] = df[col].str.replace('\s{2,}', ' ') # remove double or more white spaces
                df[col] = df[col].str.encode('ascii', 'ignore').str.decode('ascii')
    return df

cord19 = load_cord19(CORD19_PATH, cols_to_keep = ['cord_uid', 'abstract', 'title'], index_col=None)
cord19.dropna(subset=['abstract'], inplace=True)
cord19.fillna('', inplace=True)
cord19.head()

,cord_uid,title,abstract
0,ug7v899j,Clinical features of culture-proven Mycoplasma...,OBJECTIVE: This retrospective chart review des...
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...,Inflammatory diseases of the respiratory tract...
2,ejv2xln0,Surfactant protein-D and pulmonary host defense,Surfactant protein-D (SP-D) participates in th...
3,2b73a28n,Role of endothelin-1 in lung disease,Endothelin-1 (ET-1) is a 21 amino acid peptide...
4,9785vg6d,Gene expression in epithelial cells in respons...,Respiratory syncytial virus (RSV) and pneumoni...


In [2]:
cord19.isnull().sum()

cord_uid    0
title       0
abstract    0
dtype: int64

**Import necessary libraries**

In [3]:
import json
import time
import os
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import tensorflow as tf
import tensorflow_hub as hub

**Step 1. Connect to ES Cloud instance and Test ES client connection**

Upload `elastic.json` which contains 

```
{
"user": "elastic", 
"password": "<password>",
"cloud_id": "<cloud_id>"
}
```

In [4]:
with open("elastic.json") as elastic_file:
    ELASTIC_SETTINGS = json.loads(elastic_file.read().strip())

In [5]:
es_client = Elasticsearch(
    cloud_id=ELASTIC_SETTINGS["cloud_id"],
    http_auth=(ELASTIC_SETTINGS["user"], ELASTIC_SETTINGS["password"]),
)
print(f'Is ES client connected ? - {es_client.ping()}')

Is ES client connected ? - True


**Step 2. Load TF Hub model**

We are interested to use sentence embeddings techniques to embed the scholarly texts. We will use the Universal Sentence Encoder from TensorFlow Hub. You can use any pretrained models that can be used to produce the representations such as Hugging Face Transformers, etc. Load the pretrained model:

In [6]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

Let's generate embeddings for a piece of abstract as an example.

In [7]:
sample_doc = cord19.iloc[2]['abstract']
print(sample_doc)

Surfactant protein-D (SP-D) participates in the innate response to inhaled microorganisms and organic antigens, and contributes to immune and inflammatory regulation within the lung. SP-D is synthesized and secreted by alveolar and bronchiolar epithelial cells, but is also expressed by epithelial cells lining various exocrine ducts and the mucosa of the gastrointestinal and genitourinary tracts. SP-D, a collagenous calcium-dependent lectin (or collectin), binds to surface glycoconjugates expressed by a wide variety of microorganisms, and to oligosaccharides associated with the surface of various complex organic antigens. SP-D also specifically interacts with glycoconjugates and other molecules expressed on the surface of macrophages, neutrophils, and lymphocytes. In addition, SP-D binds to specific surfactant-associated lipids and can influence the organization of lipid mixtures containing phosphatidylinositol in vitro. Consistent with these diverse in vitro activities is the observati

In [8]:
embeddings = embed([sample_doc])
print(embeddings.shape)

(1, 512)


You can see that the embedding generated for the piece of text is of size (1, 512). These vector representation will be stored into Elasticsearch using a dense_vector type including the original pieces of text.

**Step 3. Index Abstract Text and Sentence Embeddings into ElasticSearch**

Setup index settings before creating it.

In [9]:
INDEX_NAME = 'abstract'
INDEX_FILE = 'abstract_settings.json'

def load_index_file(INDEX_FILE):
    """Loads index file"""
    with open(INDEX_FILE, encoding='utf-8') as index_file:
        source = index_file.read().strip()
    return source

def embed_text(text):
    """Converts text to sentence embeddings"""
    vectors = embed(text)
    return [vector.numpy().tolist() for vector in vectors]

In [10]:
from pprint import pprint
pprint(json.loads(load_index_file(INDEX_FILE)))

{'mappings': {'_source': {'enabled': 'true'},
              'dynamic': 'true',
              'properties': {'abstract': {'type': 'text'},
                             'title': {'fields': {'keyword': {'type': 'keyword'}},
                                       'type': 'text'},
                             'title_vector': {'dims': 512,
                                              'type': 'dense_vector'}}},
 'settings': {'number_of_replicas': 1, 'number_of_shards': 2}}


#### a) Create Elasticsearch Index

In [11]:
def create_index(es_client):
    """ Creates an Elasticsearch index."""
    is_created = False
    # Index settings
    settings = load_index_file(INDEX_FILE)
    try:
        if es_client.indices.exists(INDEX_NAME):
            es_client.indices.delete(index=INDEX_NAME, ignore=[404])
            print(f'\t[*] Deleting existing {INDEX_NAME} index...')
        print(f'\t[*] Creating {INDEX_NAME} index...')
        es_client.indices.create(index=INDEX_NAME, body=settings)
        is_created = True
        print(f'\t[+] {INDEX_NAME} index created successfully.')
    except Exception as ex:
        print(str(ex))
        print(f'\t[X] Failed to create {INDEX_NAME} index.')
    return is_created
create_index(es_client)

	[*] Deleting existing abstract index...
	[*] Creating abstract index...
	[+] abstract index created successfully.


True

#### b) Index data in bulk

In [12]:
def index_data(es_client, df, BATCH_SIZE=1000):
    """ Indexs all the rows in data (python questions)."""
    docs = []
    count = 0
    for _, row in df.iterrows():
        json_object = {}
        json_object['cord_uid'] = row['cord_uid']
        json_object['title'] = row['title']
        json_object['abstract'] = row['abstract']
        docs.append(json_object)
        count += 1
        
        if count % BATCH_SIZE == 0:
            index_batch(docs)
            docs = []
            print('Indexed {} documents.'.format(count))
    if docs:
        index_batch(docs)
        print('Indexed {} documents.'.format(count))
    
    es_client.indices.refresh(index=INDEX_NAME)
    print("Done indexing.")

def index_batch(docs):
    titles = [doc['title'] for doc in docs]
    title_vectors = embed_text(titles)

    requests = []
    for i, doc in enumerate(docs):
        request = doc
        request["_op_type"] = "index"
        request["_index"] = INDEX_NAME
        request["title"] = doc["title"]
        request["abstract"] = doc['abstract']
        request["title_vector"] = title_vectors[i]
        requests.append(request)
    bulk(es_client, requests)

In [13]:
%%time
index_data(es_client, cord19)

Indexed 1000 documents.
Indexed 2000 documents.
Indexed 3000 documents.
Indexed 4000 documents.
Indexed 5000 documents.
Indexed 6000 documents.
Indexed 7000 documents.
Indexed 8000 documents.
Indexed 9000 documents.
Indexed 10000 documents.
Indexed 11000 documents.
Indexed 12000 documents.
Indexed 13000 documents.
Indexed 14000 documents.
Indexed 15000 documents.
Indexed 16000 documents.
Indexed 17000 documents.
Indexed 18000 documents.
Indexed 19000 documents.
Indexed 20000 documents.
Indexed 21000 documents.
Indexed 22000 documents.
Indexed 23000 documents.
Indexed 24000 documents.
Indexed 25000 documents.
Indexed 26000 documents.
Indexed 27000 documents.
Indexed 28000 documents.
Indexed 29000 documents.
Indexed 30000 documents.
Indexed 31000 documents.
Indexed 32000 documents.
Indexed 33000 documents.
Indexed 34000 documents.
Indexed 35000 documents.
Indexed 36000 documents.
Indexed 37000 documents.
Indexed 38000 documents.
Indexed 39000 documents.
Indexed 40000 documents.
Indexed 4

### Load Topics


In [33]:
def load_queries(input_fpath: Path, dtype: str = 'csv', cols_to_keep=['topic-id', 'query', 'question'], index_col=['topic-id']) -> pd.DataFrame:
    """Loads queries file and returns it as pandas data frame
    """
    if dtype == 'csv':
        df = pd.read_csv(input_fpath, quotechar='"', index_col=index_col, usecols=cols_to_keep)
        # for each column
        for col in df.columns:
            # check if the columns contains string data
            if pd.api.types.is_string_dtype(df[col]):
                df[col] = df[col].str.strip() # removes front and end white spaces
                df[col] = df[col].str.replace('\s{2,}', ' ') # remove double or more white spaces
    return df

QUERY_FPATH = Path('../data/CORD-19/CORD-19/topics-rnd3.csv')
query_df = load_queries(QUERY_FPATH)
query_df['query+question'] = query_df['query'] + ' ' + query_df['question']
query_df.head()

,query,question,query+question
topic-id,,,
1,coronavirus origin,what is the origin of COVID-19,coronavirus origin what is the origin of COVID-19
2,coronavirus response to weather changes,how does the coronavirus respond to changes in...,coronavirus response to weather changes how do...
3,coronavirus immunity,will SARS-CoV2 infected people develop immunit...,coronavirus immunity will SARS-CoV2 infected p...
4,how do people die from the coronavirus,what causes death from Covid-19?,how do people die from the coronavirus what ca...
5,animal models of COVID-19,what drugs have been active against SARS-CoV o...,animal models of COVID-19 what drugs have been...


### Test Query

Let's write some helper functions below to query newly built Elastic Search index.

In [34]:
def search_on_abstract(query, qid, run_name, es_client, top_k=10, verbose=False):
    """ Searches the query and finds the best matches using elasticsearch."""
    # 1. create trec-covid template
    template = "{} Q0 {} {} {:.6f} {}\n"
    # 2. create ES search query
    search = {
        "size": top_k, 
        "query": {"match": {"abstract": query}},
        "_source": {"includes": ["cord_uid", "title"]}
    }
    response = es_client.search(
        index=INDEX_NAME,
        body=json.dumps(search)
    )
    ranked_lists = []
    for rank, hit in enumerate(response["hits"]["hits"]):
        cord_uid = hit["_source"]["cord_uid"]
        score = hit["_score"]
        title = hit["_source"]["title"]
        ranked_lists.append(template.format(qid, cord_uid, rank+1, score, run_name))
        if verbose:
            print("\tcord_id: {}".format(cord_uid))
            print("\ttitle: {}".format(title))
            print("\tscore: {}".format(score))
            print()
    return ranked_lists

In [43]:
qid = 1
query = query_df.loc[qid]['query']
tmp = search_on_abstract(query, qid, 'elastic_search_baseline', es_client, top_k=10, verbose=True)

	cord_id: 8ccl9aui
	title: Mosaic evolution of the severe acute respiratory syndrome coronavirus.
	score: 9.693761

	cord_id: llv3cvdr
	title: Using the spike protein feature to predict infection risk and monitor the evolutionary dynamic of coronavirus
	score: 9.092906

	cord_id: 4dtk1kyh
	title: Origin of Novel Coronavirus (COVID-19): A Computational Biology Study using Artificial Intelligence
	score: 8.979626

	cord_id: ab757i3f
	title: Emergence of a Novel Coronavirus, Severe Acute Respiratory Syndrome Coronavirus 2: Biology and Therapeutic Options
	score: 8.883345

	cord_id: d6by9p41
	title: Emergence of a Novel Coronavirus, Severe Acute Respiratory Syndrome Coronavirus 2: Biology and Therapeutic Options
	score: 8.883345

	cord_id: u65mey2z
	title: Classical Coronaviruses
	score: 8.824791

	cord_id: hfkzu18p
	title: SARS-CoV-2 and COVID-19: The most important research questions
	score: 8.689409

	cord_id: uqls3p01
	title: Genomic variance of the 2019-nCoV coronavirus
	score: 8.6462

In [44]:
def write_results(out_fpath, query_df, query_txt_col, es_client, run_name, top_k=1000):
    """Writes ranked results from elastic search results to txt file."""
    with open(out_fpath, 'w', encoding='utf-8') as writer:
        for idx, row in query_df.iterrows():
            qid = idx
            query = row[query_txt_col]
            ranked_lists = search_on_abstract(query, qid, run_name, es_client, top_k=top_k)
            writer.writelines(ranked_lists)
    print(f"Wrote file @ {out_fpath}\n")

### a) Search on `abstract` using `query` only

In [46]:
%%time
run_name = 'elasticsearch_baseline_abstract_query'
query_txt_col = 'query'
out_fpath = Path('../data/output') / f'{run_name}.txt'
write_results(out_fpath, query_df, query_txt_col, es_client, run_name)

Wrote file @ ../data/output/elasticsearch_baseline_abstract_query.txt

CPU times: user 309 ms, sys: 8.75 ms, total: 318 ms
Wall time: 23.9 s


In [47]:
run_name = "elasticsearch_baseline_abstract_query"
path_to_qrel_file = "../data/qrels/qrels-covid_d3_j0.5-3.txt"
path_to_result_file = f"../data/output/{run_name}.txt"
output_result_path = f"../data/results/{run_name}_trec_eval.txt"
os.system("trec_eval -c -m all_trec {} {} > {}".format(path_to_qrel_file, path_to_result_file, output_result_path))
with open(output_result_path, encoding='utf-8') as f:
    print(f.read())

runid                 	all	elasticsearch_baseline_abstract_query
num_q                 	all	40
num_ret               	all	40000
num_rel               	all	10001
num_rel_ret           	all	3371
map                   	all	0.1285
gm_map                	all	0.0684
Rprec                 	all	0.2087
bpref                 	all	0.3039
recip_rank            	all	0.7259
iprec_at_recall_0.00  	all	0.7757
iprec_at_recall_0.10  	all	0.3716
iprec_at_recall_0.20  	all	0.2641
iprec_at_recall_0.30  	all	0.1685
iprec_at_recall_0.40  	all	0.1018
iprec_at_recall_0.50  	all	0.0738
iprec_at_recall_0.60  	all	0.0325
iprec_at_recall_0.70  	all	0.0037
iprec_at_recall_0.80  	all	0.0000
iprec_at_recall_0.90  	all	0.0000
iprec_at_recall_1.00  	all	0.0000
P_5                   	all	0.5500
P_10                  	all	0.5350
P_15                  	all	0.4833
P_20                  	all	0.4537
P_30                  	all	0.4167
P_100                 	all	0.2905
P_200                 	all	0.2174
P_500                 	al

**Key Metrics**

- `MAP` - 0.1285
- `NDCG@10` - 0.4569
- `P@5` - 0.5500
- `R@1000` - 0.3604


### b) Search on `abstract` using `question + query`

In [48]:
%%time
run_name = 'elasticsearch_baseline_abstract_query_question'
query_txt_col = 'query+question'
out_fpath = Path('../data/output') / f'{run_name}.txt'
write_results(out_fpath, query_df, query_txt_col, es_client, run_name)

Wrote file @ ../data/output/elasticsearch_baseline_abstract_query_question.txt

CPU times: user 314 ms, sys: 13.2 ms, total: 327 ms
Wall time: 24.1 s


In [50]:
run_name = "elasticsearch_baseline_abstract_query_question"
path_to_qrel_file = "../data/qrels/qrels-covid_d3_j0.5-3.txt"
path_to_result_file = f"../data/output/{run_name}.txt"
output_result_path = f"../data/results/{run_name}_trec_eval.txt"
os.system("trec_eval -c -m all_trec {} {} > {}".format(path_to_qrel_file, path_to_result_file, output_result_path))
with open(output_result_path, encoding='utf-8') as f:
    print(f.read())

runid                 	all	elasticsearch_baseline_abstract_query_question
num_q                 	all	40
num_ret               	all	40000
num_rel               	all	10001
num_rel_ret           	all	4113
map                   	all	0.1679
gm_map                	all	0.1180
Rprec                 	all	0.2522
bpref                 	all	0.3556
recip_rank            	all	0.7923
iprec_at_recall_0.00  	all	0.8408
iprec_at_recall_0.10  	all	0.4761
iprec_at_recall_0.20  	all	0.3490
iprec_at_recall_0.30  	all	0.2489
iprec_at_recall_0.40  	all	0.1536
iprec_at_recall_0.50  	all	0.0769
iprec_at_recall_0.60  	all	0.0339
iprec_at_recall_0.70  	all	0.0040
iprec_at_recall_0.80  	all	0.0000
iprec_at_recall_0.90  	all	0.0000
iprec_at_recall_1.00  	all	0.0000
P_5                   	all	0.6300
P_10                  	all	0.5975
P_15                  	all	0.5817
P_20                  	all	0.5538
P_30                  	all	0.5158
P_100                 	all	0.3552
P_200                 	all	0.2650
P_500           

**Key Metrics**

- `MAP` - 0.1679
- `NDCG@10` - 0.5350
- `P@5` - 0.6300
- `R@1000` - 0.4270

vs. best TF-IDF results,

**TF-IDF `abstract` and `query+question`**

- `MAP` - 0.1596
- `NDCG@10` - 0.4311
- `P@5` - 0.5450
- `R@1000` - 0.4436

Wow! Significant improvement over TF-IDF baseline, while recall slightly went down, precision improved and NDCG also improved. Next let's compare it against USE encoded `title vectors`.

### c) Search using `title` vectors + `query + question`

In [61]:
def cosinesearch_on_title_vectors(query, qid, run_name, es_client, top_k=10, verbose=False):
    """ Searches using title vectors and finds the best matches using elasticsearch."""
    # 1. create trec-covid template
    template = "{} Q0 {} {} {:.6f} {}\n"
    # 2. get query vector
    query_vector = embed_text([query])[0]
    # 3. create script query for cosine search
    script_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, doc['title_vector']) + 1.0",
                "params": {"query_vector": query_vector}
            }
        }
    }
    # 4. create ES search query
    search = {
        "size": top_k, 
        "query": script_query,
        "_source": {"includes": ["cord_uid", "title"]}
    }
    # 5. search 
    response = es_client.search(
        index=INDEX_NAME,
        body=json.dumps(search)
    )
    # 6. parse results
    ranked_lists = []
    for rank, hit in enumerate(response["hits"]["hits"]):
        cord_uid = hit["_source"]["cord_uid"]
        score = hit["_score"]
        title = hit["_source"]["title"]
        ranked_lists.append(template.format(qid, cord_uid, rank+1, score, run_name))
        if verbose:
            print("\tcord_id: {}".format(cord_uid))
            print("\ttitle: {}".format(title))
            print("\tscore: {}".format(score))
            print()
    return ranked_lists

In [57]:
qid = 1
query = query_df.loc[qid]['query+question']
tmp = cosinesearch_on_title_vectors(query, qid, 'elastic_search_baseline', es_client, top_k=10, verbose=True)

	cord_id: dv9m19yk
	title: [What is the origin of SARS-CoV-2?]
	score: 1.5443752

	cord_id: 8zwsi4nk
	title: Date of origin of the SARS coronavirus strains
	score: 1.54244

	cord_id: a8voc4n9
	title: Date of origin of the SARS coronavirus strains.
	score: 1.54244

	cord_id: 9jons9u9
	title: COVID-19 (Coronavirus)
	score: 1.5143869

	cord_id: azdrlir3
	title: COVID-19 (Coronavirus)
	score: 1.5143869

	cord_id: h8ahn8fw
	title: Origin and evolution of the 2019 novel coronavirus
	score: 1.5045092

	cord_id: xdzbwa6z
	title: Origins of peptidases
	score: 1.4890406

	cord_id: 4dtk1kyh
	title: Origin of Novel Coronavirus (COVID-19): A Computational Biology Study using Artificial Intelligence
	score: 1.4741515

	cord_id: q089uoz2
	title: The origin of HIV-1, the AIDS virus.
	score: 1.4706244

	cord_id: 8na0nn5s
	title: The origin of HIV-1, the AIDS virus
	score: 1.4706244



In [58]:
def write_results(out_fpath, query_df, query_txt_col, es_client, run_name, top_k=1000):
    """Writes ranked results from elastic search results to txt file."""
    with open(out_fpath, 'w', encoding='utf-8') as writer:
        for idx, row in query_df.iterrows():
            qid = idx
            query = row[query_txt_col]
            ranked_lists = cosinesearch_on_title_vectors(query, qid, run_name, es_client, top_k=top_k)
            writer.writelines(ranked_lists)
    print(f"Wrote file @ {out_fpath}\n")

In [59]:
%%time
run_name = 'elasticsearch_baseline_title_vectors_query_question'
query_txt_col = 'query+question'
out_fpath = Path('../data/output') / f'{run_name}.txt'
write_results(out_fpath, query_df, query_txt_col, es_client, run_name)

Wrote file @ ../data/output/elasticsearch_baseline_title_vectors_query_question.txt

CPU times: user 7.2 s, sys: 2.17 s, total: 9.37 s
Wall time: 41.2 s


In [60]:
run_name = "elasticsearch_baseline_title_vectors_query_question"
path_to_qrel_file = "../data/qrels/qrels-covid_d3_j0.5-3.txt"
path_to_result_file = f"../data/output/{run_name}.txt"
output_result_path = f"../data/results/{run_name}_trec_eval.txt"
os.system("trec_eval -c -m all_trec {} {} > {}".format(path_to_qrel_file, path_to_result_file, output_result_path))
with open(output_result_path, encoding='utf-8') as f:
    print(f.read())

runid                 	all	elasticsearch_baseline_title_vectors_query_question
num_q                 	all	40
num_ret               	all	40000
num_rel               	all	10001
num_rel_ret           	all	2212
map                   	all	0.0518
gm_map                	all	0.0255
Rprec                 	all	0.1180
bpref                 	all	0.2092
recip_rank            	all	0.6408
iprec_at_recall_0.00  	all	0.6762
iprec_at_recall_0.10  	all	0.1839
iprec_at_recall_0.20  	all	0.0830
iprec_at_recall_0.30  	all	0.0338
iprec_at_recall_0.40  	all	0.0114
iprec_at_recall_0.50  	all	0.0018
iprec_at_recall_0.60  	all	0.0000
iprec_at_recall_0.70  	all	0.0000
iprec_at_recall_0.80  	all	0.0000
iprec_at_recall_0.90  	all	0.0000
iprec_at_recall_1.00  	all	0.0000
P_5                   	all	0.3950
P_10                  	all	0.3400
P_15                  	all	0.3117
P_20                  	all	0.2925
P_30                  	all	0.2600
P_100                 	all	0.1697
P_200                 	all	0.1235
P_500      

**Key Metrics**

- `MAP` - 0.0518
- `NDCG@10` - 0.3377
- `P@5` - 0.3950
- `R@1000` - 0.2347

Results are poor compared to Full-text search of Elasticsearch. One reason maybe it's `title` might be enough to get the proper context for query vector to form, also pre-trained embeddings usually have poor results on specific literature like COVID-19.
